In [ ]:
# Change working directory to project root.
import os
if os.getcwd().endswith('qa'):
    print(os.getcwd())
    os.chdir('../..')
print(os.getcwd())

In [ ]:
# Test whether progress bars work.
from tqdm.auto import tqdm
_ = list(tqdm(range(1)))

In [ ]:
import awe.utils
import awe.qa.trainer
import awe.qa.eval
import awe.training.logging
awe.utils.reload('awe.qa', 'awe.training', 'awe.utils')

In [ ]:
version = awe.training.logging.Version.get_latest()
version.exists(), version

In [ ]:
*_, checkpoint = version.get_checkpoints()
checkpoint.file_path

In [ ]:
params = awe.qa.trainer.TrainerParams.load_version(version)
params

In [ ]:
# Preserve previously loaded pipeline for faster iteration.
pipeline = trainer.pipeline if 'trainer' in globals() else None

In [ ]:
trainer = awe.qa.trainer.Trainer(params)

In [ ]:
# Reload params.
trainer.params = params

In [ ]:
# Reload evaluator.
trainer.evaluator = awe.qa.eval.ModelEvaluator(trainer.label_map)

In [ ]:
if pipeline is not None:
    trainer.pipeline = pipeline
trainer.load_pipeline()

In [ ]:
trainer.load_data()

In [ ]:
trainer.create_model()

In [ ]:
trainer.restore(checkpoint)

In [ ]:
trainer.validate(awe.qa.trainer.RunInput(trainer.create_dataloader(trainer.val_pages)))

In [ ]:
[p.file_path for p in trainer.val_pages[:2]]

In [ ]:
preds = trainer.predict(awe.qa.trainer.RunInput(trainer.create_dataloader(trainer.val_pages[:2])))
trainer.decode(preds)

In [ ]:
trainer.validate(awe.qa.trainer.RunInput(trainer.create_dataloader(trainer.train_pages[:2])))

In [ ]:
[p.file_path for p in trainer.train_pages[:2]]

In [ ]:
preds = trainer.predict(awe.qa.trainer.RunInput(trainer.create_dataloader(trainer.train_pages[:2])))
trainer.decode(preds)